In [2]:
using DataFrames, CSV, Statistics

# Find Time Spent on Each Module

In [2]:
# Function that returns true only on df rows that match changes in module. The STATE START flag also applies to changes in clip, so the demosequence entity is filtered out
function app_step_filter(row)
    (row.tag == "STATE START" && (row.entity == "LOGINMANAGER" || row.entity == "LABMANAGER")) ||
    (row.tag == "SUBMIT")
end

app_step_filter (generic function with 1 method)

### Create Mapping from phase name to index in array
Use a dictionary to link the phase change message to what index in the list the time should be stored at

In [24]:
phase_index_mapping = let 
    # Create a list of the message values of each change in state log, in order
    phases = ["Introduction", "Placement", "Pin Entry", "Authentication", "Lab Selection", "Lab Initiation",
          "Module 0", "Module 1", "Module 2", "Module 3", "Module 4", "Module 5", "Module 6", "Module 7",
          "SUBMIT"]
    # iterate through the list, map from string value to list index using the dictionary
    mapping = Dict()
    for (index, item) in enumerate(phases)
        mapping[item] = index
    end
    # return the generated mapping from the local scope
    mapping
end
show(phase_index_mapping)

Dict{Any, Any}("Module 7" => 14, "Authentication" => 4, "Module 3" => 10, "Module 1" => 8, "Module 5" => 12, "Introduction" => 1, "Module 0" => 7, "Lab Initiation" => 6, "Module 2" => 9, "Placement" => 2, "Module 6" => 13, "Module 4" => 11, "Lab Selection" => 5, "SUBMIT" => 15, "Pin Entry" => 3)

### Create DataFrame of Time Spent on Each Module

In [26]:
### Create the empty DataFrame
# Using a let block means the intermediate vector names do not persist
# And they don't clutter the namespace
spent_time_per_section = let 
    # Create a vector of empty vectors of appropriate types for each column
    # (If you don't explicitly create a vectory of type Any, it typepromotes the UInt8 to a Float64)
    col_types = push!(Vector{Any}(), UInt8[], fill(Float32[], 15)...)
    # Create a vector of names for each column
    col_names = ["User ID"; "Introduction"; "Placement"; "Pin Entry"; "Authentication"; "Lab Selection"; "Lab Initiation"; ["module$(x)" for x in 0:7]; "Total"]
    # Construct the empty data frame
    DataFrame(col_types, col_names)
end

### Construct the Dataframes
# Loop through each user's logs
for filename in readdir(raw"..\Data\Dataframes\Logs", join=true)
    # Get the dataframe from the file
    df = DataFrame(CSV.File(filename))
    # Get the relevant rows
    step_change_logs = filter(app_step_filter, df)
    # Get the id of the user (No handling of Guest for now)
    id = parse(Int, match(r"(\d{2,3})_logs.csv$", filename)[1])
    # Initialize empty list of times for this user
    times_vec = Vector{Float64}(undef, 14)
    # Value to store the reltime from the last row
    last_time = 0
    # index into the list matching the phase that was completed
    step_index = 1

    # Start on the second row, the first row is just the start of the Introduction, and leave out the last row, handled after
    for row in eachrow(step_change_logs)[2:end-1,:]
        # cur row reltime - last row reltime is the total time spent on last row's state
        times_vec[step_index] = row.reltime - last_time
        # set the last time to the current row's time 
        last_time = row.reltime
        # use the mapping to find the next index. This handles if a user goes back and forth
        step_index = phase_index_mapping[strip(row.message)]
    end

    # Find the total time spend in the app
    total_time = last(step_change_logs).reltime

    # Create a tuple holding all the values for this user, append it as a row to the df
    push!(spent_time_per_section, tuple(id, times_vec..., total_time))
end

# Sort the table by user ID, since the dictionary does not supply them in order
sort!(spent_time_per_section, "User ID")

# Quick check to see if the whole is greater than the sum of its parts
print("Sum of individual items: $(sum(spent_time_per_section[1,2:end-1])), total time from log: $(spent_time_per_section[1,:Total])")

show(first(spent_time_per_section, 8),allcols=true)

Sum of individual items: 95.600006, total time from log: 97.58×16 DataFrame
 Row │ User ID  Introduction  Placement  Pin Entry  Authentication  Lab Selection  Lab Initiation  module0  module1  module2  module3  module4  module5  module6  module7  Total   
     │ UInt8    Float32       Float32    Float32    Float32         Float32        Float32         Float32  Float32  Float32  Float32  Float32  Float32  Float32  Float32  Float32 
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │      88          8.68       3.14       5.54             0.0           3.95            9.27     9.98     8.96     6.59     7.45     8.57    11.82     9.39     2.26    97.5
   2 │     102          8.68      33.86      45.94             0.0          16.17            6.48   848.87   399.21   385.71    65.58   250.26   405.45    96.35    84.35  2650.48
   3 │     103          8.6

In [31]:
## export the df
CSV.write("../Data/Dataframes/time_spent_per_section.csv", spent_time_per_section)

"../Data/Dataframes/time_spent_per_section.csv"

### Create Dataframe of Start Times of Each Section

In [3]:
# If spent_time_per_section isn't defined, read in the df from the csv file
if !@isdefined spent_time_per_section
    spent_time_per_section = DataFrame(CSV.File("../Data/Dataframes/time_spent_per_section.csv"))
end

,User ID,Introduction,Placement,Pin Entry,Authentication,Lab Selection,Lab Initiation
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,88,8.68,3.14,5.54,0.0,3.95,9.27
2,102,8.68,33.86,45.94,0.0,16.17,6.48
3,103,8.68,32.54,16.74,0.0,2.63,14.85
4,105,8.68,39.1,18.8,0.0,5.33,6.52
5,108,8.68,67.09,34.8,0.0,5.42,6.79
6,110,8.68,36.03,16.96,0.0,12.35,9.01
7,111,8.67,30.81,10.82,0.0,9.5,6.56
8,112,8.68,28.49,55.59,0.0,33.12,7.26
9,113,8.68,66.94,23.06,0.0,26.91,6.69


In [29]:
# Define a function that takes a named tuple of the row of times spent per sections
# and converts it to the start time of each section, converts it to a named tuple with their column names
find_start_times(x) = (; zip(keys(x), [0 + sum(collect(x)[1:i], init=Float32(0)) for i in 0:length(x)-1])...)

start_times_per_section = select(
    spent_time_per_section,
    "User ID",
    AsTable(Between("Introduction","module7")) => ByRow(find_start_times) => AsTable,
    renamecols=false
)
start_times_per_section[:,"Submit"] = spent_time_per_section[:,"Total"]

show(start_times_per_section, allcols=true)

39×16 DataFrame
 Row │ User ID  Introduction  Placement  Pin Entry  Authentication  Lab Selection  Lab Initiation  module0  module1  module2  module3  module4  module5  module6  module7  Submit  
     │ UInt8    Float32       Float32    Float32    Float32         Float32        Float32         Float32  Float32  Float32  Float32  Float32  Float32  Float32  Float32  Float32 
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │      88           0.0       8.68      11.82           17.36          17.36           21.31    30.58    40.56    49.52    56.11    63.56    72.13    83.95    93.34    97.5
   2 │     102           0.0       8.68      42.54           88.48          88.48          104.65   111.13   960.0   1359.21  1744.92  1810.5   2060.76  2466.21  2562.56  2650.48
   3 │     103           0.0       8.68      41.22           57.96          57.96      

In [30]:
# Export CSV
CSV.write("../Data/Dataframes/start_time_per_section.csv", start_times_per_section)

"../Data/Dataframes/start_time_per_section.csv"

### Collect when users spend more than 75% more time than average, and when they spend less than 75% less time than average on a step

In [4]:
# If spent_time_per_section isn't defined, read in the df from the csv file
if !@isdefined spent_time_per_section
    spent_time_per_section = DataFrame(CSV.File("../Data/Dataframes/time_spent_per_section.csv"))
end


In [36]:
(below_average, above_average) = let
    averages = Vector{Float32}(undef, length(spent_time_per_section[1,2:12]))
    # calculate the average time spent on each module9
    for (index, col) in enumerate(eachcol(spent_time_per_section[!,2:12]))
        averages[index] = mean(col)
    end

    # go through each user and print flag when they spend significantly above average or below average time on a module.
    colnames = names(spent_time_per_section)
    low = let 
        col_types = push!(Vector{Any}(), UInt8[], String[], Float32[], Float32[])
        col_names = ["User ID"; "Module"; "Time on Module"; "Average"]
        DataFrame(col_types, col_names)
    end
    high = let 
        col_types = push!(Vector{Any}(), UInt8[], String[], Float32[], Float32[])
        col_names = ["User ID"; "Module"; "Time on Module"; "Average"]
        DataFrame(col_types, col_names)
    end
    for (uindex,user) in enumerate(eachrow(spent_time_per_section)), (cindex, val) in enumerate(user[2:12])
        if val > 1.75*averages[cindex]
            push!(high, (user[1], colnames[1+cindex], val, averages[cindex]))
        end
        if val < 0.25*averages[cindex]
            push!(low, (user[1], colnames[1+cindex], val, averages[cindex]))
        end
    end
    low, high
end

(18×4 DataFrame
 Row │ User ID  Module     Time on Module  Average  
     │ UInt8    String     Float32         Float32  
─────┼──────────────────────────────────────────────
   1 │      88  Placement            3.14   42.9872
   2 │      88  Pin Entry            5.54   30.1177
   3 │      88  module0              9.98  618.564
   4 │      88  module1              8.96  461.963
   5 │      88  module2              6.59  341.219
   6 │      88  module3              7.45  348.642
   7 │      88  module4              8.57  326.588
   8 │     102  module3             65.58  348.642
  ⋮  │    ⋮         ⋮            ⋮            ⋮
  12 │     111  module1            100.25  461.963
  13 │     112  module2             70.46  341.219
  14 │     117  Placement            0.74   42.9872
  15 │     122  module1             68.03  461.963
  16 │     128  module2             40.43  341.219
  17 │     145  module3             77.63  348.642
  18 │     151  module3             10.26  348.642
         

In [37]:
# Export CSV
CSV.write("../Data/Dataframes/spent_time_below_average.csv", below_average)
CSV.write("../Data/Dataframes/spent_time_above_average.csv", above_average)

"../Data/Dataframes/spent_time_above_average.csv"